# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [1]:
import numpy as np


### ESOINN node class
* feature_vector - weights
* accamulate_signals – number of signals
* total_points – points $\neq$ number of signals
* density – mean accumulated signals
* subclass_id – mark for subclass

In [2]:
class ESOINN_Node:
    def __init__(self, feature_vector=()):
        self.feature_vector = np.array(feature_vector)  
        self.accamulate_signals = 0
        self.total_points = 0
        self.density = 0
        self.subclass_id = -1
    
    def update_accamulate_signals(self, n=1):
        self.accamulate_signals += 1

### ESOINN Neural Network class
To start lerning use: `fit()` method, for clasterization use `predict()`.

Hiperparams:
* C1, C2 – coefficents for noise deletion.
* learning_step
* max_age – for edges.
* forget – specify which N is used in density calculation.
* metrics – lambda(x, y, axis)

Params:
* ids – last given id for nodes.

In [72]:
class ESOINN_NN:
    def __init__(self, init_nodes, C1=0.001, C2=1, learning_step=200, max_age=50, 
                 metrics=lambda x,y,axis=0: np.sqrt(np.sum(np.square(np.array(x) - np.array(y)), axis=axis)),
                 forget=False, relation_coeff=1):
        self.C1 = C1
        self.C2 = C2
        self.learning_step = learning_step
        self.max_age = max_age
        self.signals_amount = 2
        self.metrics = metrics
        self.forget = forget
        self.unique_id = 2
        self.rc = relation_coeff
        
        self.nodes = {i: ESOINN_Node(init_nodes[i]) for i in (0, 1)}
        self.neighbors = {}  # keys = id, values = sets of neighbors' ids
        self.edges = {}  # keys = tuples(2), where t[0] < t[1], value = age/None
    
    def fit(self, input_signal):
        self.signals_amount += 1
        
        winners_ids, distances = self.find_winners(input_signal)
        thresholds = [self.calc_threshold(input_signal, winners_ids[i]) for i in (0, 1)]
        if distances[0] > thresholds[0] or distances[1] > thresholds[1]:
            self.create_node(input_signal)
            return
        
        self.update_edges_age(winners_ids[0])
        self.build_connection(winners_ids)
        
        self.update_density(winners_ids[0])
        
        self.nodes[winners_ids[0]].update_accamulate_signals()
        
        self.update_feature_vector(winners_ids[0])
        
        self.remove_old_ages()
        
        if self.signals_amount % self.learning_step == 0:
            self.update_topology()
    
    def find_winners(self, input_signal):
        # @fixme: inf coef and separate variables for each winner
        first_winner = float('inf')
        first_winner_id = -1
        second_winner = float('inf')
        second_winner_id = -1
        for node_id in self.nodes:
            dist = self.metrics(input_signal, self.nodes[node_id].feature_vector)
            if dist <= first_winner:
                first_winner, second_winner = dist, first_winner
                second_winner_id = first_winner_id
                first_winner_id = node_id
            elif dist < second_winner:
                second_winner = dist
                second_winner_id = node_id
        return [first_winner_id, second_winner_id], [first_winner, second_winner]
    
    def find_neighbors(start_node_id, depth=1):
        visited = {start_node_id}
        queue = list(self.neighbors.get(start, set()) - visited)
        while depth:
            depth -= 1
            for vertex in queue.copy():  # @fixme: do not use copy!
                visited.add(vertex)
                queue.extend([node for node in self.neighbors[vertex] - visited if node not in visited])
        return visited - {start_node_id}    
    
    def calc_threshold(self, input_signal, winner_id):
        neighbors = self.neighbors.get(winner_id, None)
        if neighbors:
            return np.max([
                self.metrics(self.nodes[winner_id].feature_vector, self.nodes[neighbor_id].feature_vector) 
                for neighbor_id in find_neighbors(winner_id, depth=self.rc)
            ])
        else:
            return self.find_winners(self.nodes[winner_id].feature_vector)[1][1]  # 'cause first winner is always current node
    
    def create_node(self, input_signal):  
        self.nodes[self.unique_id] = ESOINN_Neuron(input_signal)
        self.unique_id += 1  # to provide unique ids for each neuron
    
    def update_edges_age(self, node_id, step=1):
        for neighbor_id in self.neighbors.get(node_id, []):
            pair_id = min(node_id, neighbor_id), max(node_id, neighbor_id)
            self.edges[pair_id] += 1
                
    # algorithm 3.2
    def build_connection(self, winners_ids):
        # case 1-2
        if self.nodes[winners_ids[0]].subclass_id == -1 or self.nodes[winners_ids[1]].subclass_id == -1 or 
        self.nodes[winners_ids[0]].subclass_id == self.nodes[winners_ids[1]].subclass_id:
            self.create_edge(winners_ids)        
        # case 3
        pass
                
# #     @fixme
#     def create_edge(self, indexes):
#         self.adjacency_matrix[indexes[0]][indexes[1]].exist = 1
#         self.adjacency_matrix[indexes[0]][indexes[1]].age = 0
#         self.adjacency_matrix[indexes[1]][indexes[0]].exist = 1
#         self.adjacency_matrix[indexes[1]][indexes[0]].age = 0
        
#     def set_subclass(self, indexes):
#         pass    
    
#     def update_density(self, node_index):
#         pass
    
#     def update_feature_vector(self, node_index):
#         pass
    
#     def remove_old_ages(self):
#         pass

#     def predict(self, input_signal):
#         pass

#     def update(self):
#         pass
    
    def current_state(self):
        return {
            'signals_amount': self.signals_amount,
            'C1': self.C1,
            'C2': self.C2,
            'lambda': self.learning_step,
            'forget': self.forget,
            'max_age': self.max_age,
            'metrics': self.metrics,
            'nodes': self.nodes,  # think about it
            'edges': self.edges
        }

SyntaxError: invalid syntax (<ipython-input-72-8b7f585184a7>, line 91)

## tests

In [70]:
input_signal = np.array([2, 2])
nn = ESOINN_NN([[1, 2], [5, 2]])

wins, dists = nn.find_winners(input_signal)

nn.calc_threshold(input_signal, wins[0])

# nn.fit(input_signal)

# for i in range(nn.nodes.shape[0]):
#     print(nn.nodes[i].feature_vector)

# nn.current_state()

4.0

In [62]:
g = {
    
}

In [18]:
def dfs(graph, start, ignore, depth=1, visited=set()):
    if depth != 0 and start not in visited:
        for vertex in graph[start] - visited:
            visited |= dfs(graph, vertex, ignore, depth-1, visited)
    visited.add(start)
    return visited - {ignore}


dfs(g, 1, 1, 1)

{2, 3}

In [15]:
def dsl(g, start, ignore, depth=1, visited=set()):
    if depth != 0:
        for vertex in g[start]:
            if vertex not in visited:
                visited.add(vertex)
                dsl(g, vertex, ignore, depth-1, visited)
    return visited - set([ignore])
    
dsl(g, 1, 1, 1)

{2, 3}

In [64]:
def bfs(graph, start, depth=1):
    visited = {start}
    queue = list(graph.get(start, set()) - visited)
    while depth:
        depth -= 1
        for vertex in queue.copy():
            visited.add(vertex)
            queue.extend([node for node in graph[vertex] - visited if node not in visited])
    return visited - {start}

bfs(g, 1, 3)

set()